## 学習と検証の実施
- 本ファイルでは、OpenPoseの学習と検証の実施を行います。AWSのGPUマシンで計算します。
- p2.xlargeで45分ほどかかります。


In [1]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.cuda.is_available()

True

## DataLoader作成

In [3]:
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

# MS COCO のファイルパスリスト作成
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(rootpath='./data/')
# maskは、人なのにアノテーションデータがない部分をmaskしてある

# Dataset作成
# 本来のtrainデータは
train_dataset = COCOkeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase='train', transform=DataTransform())

# DataLoader作成
batch_size = 32
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 辞書型変数にまとめる
dataloaders_dict = {'train': train_dataloader, 'val': None}

In [4]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()

## 損失関数の定義
OpenPoseの損失関数はheatmapsとPAFｓ，　それぞれについて正解アノテーションデータとの回帰の誤差<br>
平均二乗誤差mseを使い、各stageのheatmapsとPAFｓの全ての誤差を足し合わせる<br>
人物が写っているが姿勢のアノテーションデータがない部分の損失は計算しない（maskされている部分)

In [5]:
class OpenPoseLoss(nn.Module):
    """
    OpenPoseの損失関数のクラス
    """
    
    def __init__(self):
        super(OpenPoseLoss, self).__init__()
        
    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        """
        損失関数の計算
        
        Parameters
        ----------------
        saved_for_loss : OpenPoseNetの出力
        
        heatmap_target : [num_batch, 19, 46, 46]
            正解のアノテーション情報
        
        heatmap_mask : [num_batch, 19, 46, 46]
            heatmap画像のマスク
        
        paf_target : [num_batch, 38, 46, 46]
            正解のPAFのアノテーション情報
        
        paf_mask : [num_batch, 38, 46, 46]
            paf画像のマスク
        
        Returns
        ----------------
        loss : テンソル
            損失の値
            
        """
        
        total_loss = 0
        
        # ステージごとにlossを計算して加算する
        for j in range(6):
            # PAFsとheatmapにおいてマスクされている部分は無視させる
            # PAFs
            pred1 = saved_for_loss[j*2] * paf_mask
            gt1 = paf_target.float() * paf_mask
            
            # heatmap
            pred2 = saved_for_loss[j*2 + 1] * heat_mask
            gt2 = heatmap_target.float() * heat_mask
            
            total_loss += F.mse_loss(pred1, gt1, reduction='mean') + \
                F.mse_loss(pred2, gt2, reduction='mean')
            
        return total_loss

# 損失関数のインスタンス生成
criterion = OpenPoseLoss()

## 学習の実施
コード書き終わったらgithubにpushしてawsで学習を実行する


In [6]:
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=0.0001)

In [9]:
# 学習させる関数

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPUが使えるか確認
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('使用デバイス：　', device)
    
    # ネットワークをGPUへ
    net.to(device)
    
    # ネットワークがある程度固定であれば高速化させる
    torch.backends.cudnn.benchmark = True
    
    # 画像の枚数
    num_train_imgs = len(dataloaders_dict['train'].dataset)
    batch_size = dataloaders_dict['train'].batch_size
    
    # イテレーションカウンタをセット
    iteration = 1
    
    # epochのループ
    for epoch in range(num_epochs):
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0
        epoch_val_loss = 0.0
        
        print('-------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------------')
        
        # epoch毎にtrainとval
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
                optimizer.zero_grad()
                print(' (train) ')
            # 検証は今回はスキップ
            else:
                continue
            
            # dataloaderからminibatchずつ取り出すループ
            for imgs, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                # minibatchのサイズが1だとバッチノーマライゼーションでエラーになるので避ける
                if imgs.size()[0] == 1:
                    continue
                
                # GPUが使えるならGPUにデータを送る
                imgs = imgs.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)
                
                # optim初期化
                optimizer.zero_grad()
                
                # 順伝播計算
                with torch.set_grad_enabled(phase=='train'):
                    # (out6_1, out6_2)は使わないので_で代替
                    _, saved_for_loss = net(imgs)
                    
                    # 毎度代入するからlossの初期化はいらないのか？
                    loss = criterion(saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask)
                    
                    # 訓練時はbackpropagation
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        if (iteration % 10 == 0):
                            # 10iterにつき1回lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec'.format(iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()
                        epoch_train_loss += loss.item()
                        iteration += 1
                    # 検証時は今回はなし
                    
            # epochのpahseごとのlossの計
        t_epoch_finish = time.time()
        print('------------')
        print('epoch {} || Epoch_Train_Loss:{:.4f} || Epoch_Val_Loss: {:.4f}'.format(epoch+1, epoch_train_loss/num_train_imgs, 0))
        print('timer : {:.4f} sec'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()
        
    # 最後にネットワークを保存する
    torch.save(net.state_dict(), 'weights/openpose_net_'+str(epoch+1)+'.pth')

In [10]:
# 学習を実行
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス：　 cuda:0
-------------------
Epoch 1/2
-------------------
 (train) 
イテレーション 10 || Loss: 0.0018 || 10iter: 89.9444 sec
イテレーション 20 || Loss: 0.0016 || 10iter: 95.5492 sec
イテレーション 30 || Loss: 0.0016 || 10iter: 92.5720 sec
イテレーション 40 || Loss: 0.0015 || 10iter: 90.6413 sec
イテレーション 50 || Loss: 0.0016 || 10iter: 90.7619 sec
イテレーション 60 || Loss: 0.0016 || 10iter: 93.1956 sec
イテレーション 70 || Loss: 0.0013 || 10iter: 91.7621 sec
イテレーション 80 || Loss: 0.0013 || 10iter: 92.2868 sec
イテレーション 90 || Loss: 0.0016 || 10iter: 90.4380 sec
イテレーション 100 || Loss: 0.0016 || 10iter: 91.8847 sec
イテレーション 110 || Loss: 0.0012 || 10iter: 93.1834 sec
イテレーション 120 || Loss: 0.0017 || 10iter: 90.7749 sec
イテレーション 130 || Loss: 0.0015 || 10iter: 89.3018 sec
イテレーション 140 || Loss: 0.0014 || 10iter: 90.6615 sec
イテレーション 150 || Loss: 0.0013 || 10iter: 91.0294 sec
------------
epoch 1 || Epoch_Train_Loss:0.0016 || Epoch_Val_Loss: 0.0000
timer : 1447.6414 sec
-------------------
Epoch 2/2
-------------------
 (train) 
イテレーション 160 

In [ ]:
# lossが最初からある程度小さいのはいいのか？学習全体を通してあまりlossが減っていない...
# お手本だとイテレーション 10 || Loss: 0.0094 ||から始まっていた